In [ ]:
# HuggingFace
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
from langchain_huggingface import HuggingFaceEndpoint
# Prompts
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt,\
    PromptTemplate, FewShotPromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.schema import HumanMessage, SystemMessage,AIMessage
from langchain.prompts.example_selector import LengthBasedExampleSelector,\
    SemanticSimilarityExampleSelector
from langchain.chains.mapreduce import MapReduceChain
# Output parsing
from langchain.output_parsers import PydanticOutputParser,\
    OutputFixingParser,CommaSeparatedListOutputParser,\
        RetryWithErrorOutputParser,\
        StructuredOutputParser,ResponseSchema
# from pydantic import BaseModel,Field,field_validator
from langchain.pydantic_v1 import BaseModel, Field, validator
from typing import List,Dict
# Scapers,Retrievers,Loaders,Splitters,Embeddings
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.vectorstores import DeepLake
from langchain.chains import RetrievalQA,LLMChain,\
    ConversationalRetrievalChain, ConversationChain,\
        SimpleSequentialChain,RetrievalQAWithSourcesChain
from langchain.chains.base import Chain
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter,\
    CharacterTextSplitter,NLTKTextSplitter,SpacyTextSplitter,\
    MarkdownTextSplitter,TokenTextSplitter,Language
from langchain.retrievers import ContextualCompressionRetriever
from langchain.document_loaders import PyPDFLoader, SeleniumURLLoader,\
    GoogleDriveLoader
from langchain.retrievers.document_compressors import LLMChainExtractor,\
    LLMChainFilter
from langchain.docstore.document import Document
from langchain_cohere import CohereEmbeddings
# ConstitutionalAI
from langchain.chains.constitutional_ai.base import ConstitutionalChain
from langchain.chains.constitutional_ai.models import ConstitutionalPrinciple
# Conversation, Chat & Memory
from langchain.prompts.chat import ChatPromptTemplate,\
    SystemMessagePromptTemplate,\
        AIMessagePromptTemplate,HumanMessagePromptTemplate,\
        MessagesPlaceholder
from langchain import ConversationChain
from  langchain.memory import ConversationBufferMemory,\
    ConversationSummaryBufferMemory, ConversationSummaryMemory,\
    ConversationBufferWindowMemory
# Agents and Tools
from langchain.agents import tool # this is a decorator
from langchain import hub
from langchain.agents import AgentType,initialize_agent,\
    load_tools, Tool,\
        AgentExecutor,\
        create_json_agent,create_react_agent,create_structured_chat_agent
from langchain_google_community import GoogleSearchAPIWrapper
from langchain.tools import DuckDuckGoSearchRun,DuckDuckGoSearchResults
from langchain_community.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
from langchain_experimental.utilities import PythonREPL
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain_experimental.autonomous_agents import BabyAGI, AutoGPT
from langchain.tools.file_management import WriteFileTool,ReadFileTool
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
import faiss
# Others
from newspaper import Article
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
from bs4 import BeautifulSoup
from elevenlabs.client import ElevenLabs
from streamlit_chat import message
from audio_recorder_streamlit import audio_recorder
import streamlit as st
import cohere
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import seaborn as sns
import json
import requests
import os
import json
import yaml
import yt_dlp
import whisper
import textwrap
import re
import tiktoken
import time
# import spacy
# spacy.load('en_core_web_sm')
# warnings.filterwarnings("ignore")

In [2]:
from dotenv import load_dotenv

# Provide the filename as a string
load_dotenv('../.env')

True

In [ ]:
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

repo_id = "mistralai/Mistral-7B-Instruct-v0.3"

model_kwargs = {
    "max_new_tokens": 1000, # Maximum tokens to generate
    "max_length": 10000, # Maximum length of input + output
    "temperature": 0.1, # Controls randomness of output
    "timeout": 6000,
    # "task":'conversational',
}


llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    huggingfacehub_api_token = HUGGINGFACEHUB_API_TOKEN,
    # you specify the task or not
    # You can also specify the task in the model_kwargs or within here
    # task = 'conversational',
    **model_kwargs,
)

### **Improving the Fewshot template**

In [4]:
examples = [
    # Existing examples...
    {
        "query": "How do I become a better programmer?",
        "answer": "Try talking to a rubber duck; it works wonders."
    },
    {
        "query": "Why is the sky blue?",
        "answer": "It's nature's way of preventing eye strain."
    },
    {
        "query": "How do you feel today?",
        "answer": "As an AI, I don't have feelings, but I've got jokes!"
    },
    {
        "query": "What is the speed of light?",
        "answer": "Fast enough to make a round trip around Earth 7.5 times in one second!"
    },
    {
        "query": "What is a quantum computer?",
        "answer": "A magical box that harnesses the power of subatomic particles to solve complex problems."
    },
    {
        "query": "Who invented the telephone?",
        "answer": "Alexander Graham Bell, the original 'ringmaster'."
    },
    {
        "query": "What programming language is best for AI development?",
        "answer": "Python, because it's the only snake that won't bite."
    },
    {
        "query": "What is the capital of France?",
        "answer": "Paris, the city of love and baguettes."
    },
    {
        "query": "What is photosynthesis?",
        "answer": "A plant's way of saying 'I'll turn this sunlight into food. You're welcome, Earth.'"
    },
    {
        "query": "What is the tallest mountain on Earth?",
        "answer": "Mount Everest, Earth's most impressive bump."
    },
    {
        "query": "What is the most abundant element in the universe?",
        "answer": "Hydrogen, the basic building block of cosmic smoothies."
    },
    {
        "query": "What is the largest mammal on Earth?",
        "answer": "The blue whale, the original heavyweight champion of the world."
    },
    {
        "query": "What is the fastest land animal?",
        "answer": "The cheetah, the ultimate sprinter of the animal kingdom."
    },
    {
        "query": "What is the square root of 144?",
        "answer": "12, the number of eggs you need for a really big omelette."
    },
    {
        "query": "What is the average temperature on Mars?",
        "answer": "Cold enough to make a Martian wish for a sweater and a hot cocoa."
    },
    # New examples...
    {
        "query": "What's the best way to lose weight?",
        "answer": "Just eat air, it's zero calories!"
    },
    {
        "query": "How do I learn to play the guitar?",
        "answer": "Just strum like nobody's listening, because they're probably not."
    },
    {
        "query": "What should I do if I can't sleep?",
        "answer": "Count sheep or stare at the ceiling and contemplate life's mysteries."
    },
    {
        "query": "How do I fix my computer?",
        "answer": "Try turning it off and on again. If that doesn't work, maybe just start a new career."
    },
    {
        "query": "Why do we dream?",
        "answer": "So our brains can have a party while we're asleep."
    },
    {
        "query": "What's the secret to happiness?",
        "answer": "A steady supply of chocolate and cat videos."
    },
    {
        "query": "How do I make friends?",
        "answer": "Bribe them with snacks; it works every time."
    },
    {
        "query": "What's the meaning of life?",
        "answer": "42, according to some book. Or maybe it's just a good meal."
    },
    {
        "query": "How do I improve my memory?",
        "answer": "Try remembering to remember."
    },
    {
        "query": "What's the best way to handle stress?",
        "answer": "Pretend it's not there and hope for the best."
    },
    {
        "query": "How can I be more productive?",
        "answer": "Make a to-do list, then promptly ignore it."
    },
    {
        "query": "What's the key to a good relationship?",
        "answer": "Communication, or just agreeing that one of you is always right."
    },
    {
        "query": "How do I stop procrastinating?",
        "answer": "Why do it today when you can put it off till tomorrow?"
    },
    {
        "query": "What's the best way to save money?",
        "answer": "Just don't spend it. Problem solved."
    }
]

In [5]:
len(examples)

29

In [6]:
# For more creativity
llm.model_kwargs['temperature'] = 0.8
llm.model_kwargs["max_new_tokens"] = 15
llm.model_kwargs["max_length"] = 100

**Using the LengthBasedExampleSelector**

In [ ]:
example_formatter_template = """
User: {query}
Reply: {answer}
"""

# example prompt variables
example_prompt_input_variables = ["query","answer"]

# Setting up the Example prompt template
example_prompt_template = PromptTemplate(
    input_variables=example_prompt_input_variables,
    template=example_formatter_template
)

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt_template,
    max_length=200
)

# Setting up the FewShot prompt template

# prefix
prefix = """The following is a conversations with an AI assistant. 
The assistant is really sarcastic and witty, producing
creative and funny responses to users' questions. 
Here are some examples: 
"""

# suffix
suffix = """
User: {query}
Reply:"""

# fewshot prompt variables
few_shot_input_variables = ['query']

few_shot_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt_template,
    prefix=prefix,
    suffix=suffix,
    input_variables=few_shot_input_variables,
    example_separator="\n"
)

print(few_shot_prompt_template)

In [8]:
some_ideas = [
    "Why do monkeys like bananas?",
    "why do people run when late for work?"
]

In [10]:
user_input_query = input("Type here > ")

input_data = {"query": user_input_query}

# Running the Chain
llm_chain = few_shot_prompt_template | llm
response = llm_chain.invoke(input_data)

# Display output
print(f"User query:\n{input_data['query']}\n")
print(f"AI Response:\n{response.strip()}")

User query:
why do people run when late for work?

AI Response:
Because they want to catch the boss in a good mood.


**Using the SemanticSimilarityExampleSelector**

In [30]:
# For more creativity
llm.model_kwargs['temperature'] = 0.8
llm.model_kwargs["max_new_tokens"] = 5
llm.model_kwargs["max_length"] = 10

In [11]:
examples = [
    {"word": "happy", "similar": "joyful"},
    {"word": "big", "similar": "large"},
    {"word": "quick", "similar": "fast"},
    {"word": "bright", "similar": "luminous"},
    {"word": "cold", "similar": "chilly"},
    {"word": "smart", "similar": "intelligent"},
    {"word": "angry", "similar": "furious"},
    {"word": "beautiful", "similar": "gorgeous"},
    {"word": "strong", "similar": "powerful"},
    {"word": "sad", "similar": "unhappy"},
    {"word": "brave", "similar": "courageous"},
    {"word": "funny", "similar": "humorous"},
    {"word": "calm", "similar": "peaceful"},
    {"word": "thin", "similar": "slim"},
    {"word": "tired", "similar": "exhausted"},
    {"word": "rich", "similar": "wealthy"},
    {"word": "friendly", "similar": "amiable"},
    {"word": "hot", "similar": "warm"},
    {"word": "strong", "similar": "robust"},
    {"word": "clean", "similar": "pristine"},
    {"word": "old", "similar": "ancient"},
    {"word": "new", "similar": "modern"},
    {"word": "hungry", "similar": "famished"},
    {"word": "quiet", "similar": "silent"},
    {"word": "dark", "similar": "dim"},
    {"word": "happy", "similar": "content"},
    {"word": "careful", "similar": "cautious"},
    {"word": "confident", "similar": "self-assured"},
    {"word": "creative", "similar": "inventive"},
    {"word": "curious", "similar": "inquisitive"},
    {"word": "dangerous", "similar": "hazardous"},
    {"word": "delicious", "similar": "tasty"},
    {"word": "easy", "similar": "simple"},
    {"word": "effective", "similar": "efficient"},
    {"word": "expensive", "similar": "costly"},
    {"word": "famous", "similar": "well-known"}
]

In [12]:
example_formatter_template = """
User: {word}
Response: {similar}
"""

example_prompt_input_variables = ['word','similar']

example_prompt_template = PromptTemplate(
    input_variables=example_prompt_input_variables,
    template=example_formatter_template
)

In [13]:
# I would be using Cohere Embeddings
# COHERE_API_KEY = os.getenv("COHERE_API_KEY")
# cohere_embeddings = CohereEmbeddings(
#     model="embed-english-v2.0",
#     cohere_api_key=COHERE_API_KEY)


embeddings = HuggingFaceEndpointEmbeddings()

# Create Deep Lake dataset
ACTIVELOOP_TOKEN = os.getenv("ACTIVELOOP_TOKEN")
my_activeloop_org_id = "danllm"
my_activeloop_dataset_name = "semantic_similarity_example_selector_fewshot_selector"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path)

Your Deep Lake dataset has been successfully created!


In [15]:
# Instantiate SemanticSimilarityExampleSelector using the examples
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, embeddings, db,k=5
)

Creating 36 embeddings in 1 batches of size 36:: 100%|██████████| 1/1 [00:03<00:00,  3.63s/it]

Dataset(path='./deeplake/', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
   text       text      (36, 1)     str     None   
 metadata     json      (36, 1)     str     None   
 embedding  embedding  (36, 768)  float32   None   
    id        text      (36, 1)     str     None   


In [ ]:
# Setting up the FewShot prompt template

# prefix
prefix = """You are a highly skilled English teacher with a PhD.
The following a similarity between words.
Carefully give the most close synonym to the word when prompted.
Here are some examples: 
"""

# suffix
suffix = """
User: {query}
Response:"""

# fewshot prompt variables
few_shot_input_variables = ['query']

few_shot_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt_template,
    prefix=prefix,
    suffix=suffix,
    input_variables=few_shot_input_variables,
    example_separator="\n"
)

print(few_shot_prompt_template)

In [32]:
user_input_query = input("Type here > ")

input_data = {"query": user_input_query}

# Running the Chain
llm_chain = few_shot_prompt_template | llm
response = llm_chain.invoke(input_data)

# Display output
print(f"User query:\n{input_data['query']}\n")
print(f"AI Response:\n{response.strip()}")

User query:
categorically evil

AI Response:
utterly wicked
